In [35]:
import os
import bs4
import re
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage,HumanMessage

load_dotenv()

True

In [36]:
os.environ['HF_API_KEY']=os.getenv('HF_API_KEY')
Groq_api_key=os.getenv('GROQ_API_KEY')

llm_model=ChatGroq(groq_api_key=Groq_api_key,model_name="llama-3.3-70b-versatile")
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x107f17820>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x12ff0d3f0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [37]:
Embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/homebrew/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:

loader = WebBaseLoader("https://www.ibm.com/think/topics/ai-agents")

docs = loader.load()
#print(docs[0].page_content)

In [39]:
raw = docs[0].page_content

# removing blank lines
cleaned_text = re.sub(r"\n\s*\n+", "\n", raw)
cleaned_text= re.sub(r"[ \t]+", " ", cleaned_text)
cleaned_text = cleaned_text.strip()

#print(cleaned_text[::])


In [40]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250)
splitted_text=text_splitter.split_text(cleaned_text)
#splitted_text

In [41]:
vector_db=Chroma.from_texts(splitted_text,embedding=Embeddings)
retriever=vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x12ff0de70>, search_kwargs={})

In [42]:
#Creating System Prompt
System_prompt = (
    "Answer the question using the provided context. "
    "If unsure, say you don't know. "
    "Limit your response to three concise sentences.\n\n"
    "{context}"
)

LLM_Prompt=ChatPromptTemplate.from_messages(
    [
        ("system",System_prompt),
        ('human','{input}'),
    ]
)

In [43]:
question_answer_chain=create_stuff_documents_chain(llm_model,LLM_Prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [44]:
response=rag_chain.invoke({'input':'what are ai agents?'})
response

{'input': 'what are ai agents?',
 'context': [Document(id='52836ccb-e3a4-4dfe-b54e-0ef99810c551', metadata={}, page_content='AI agents\n What are AI agents?\nFrom monolithic models to compound AI systems, discover how AI agents integrate with databases and external tools to enhance problem-solving capabilities and adaptability.\nLearn more\n Reasoning paradigms\r\nThere is not one standard architecture for building AI agents. Several paradigms exist for solving multistep problems.\nReAct (reasoning and action)\nWith the ReAct paradigm, we can instruct agents to "think" and plan after each action taken and with each tool response to decide which tool to use next. These Think-Act-Observe loops are used to solve problems step by step and iteratively improve upon responses.\nThrough the prompt structure, agents can be instructed to reason slowly and to display each "thought".4\xa0The agent\'s verbal reasoning gives insight into how responses are formulated. In this framework, agents contin

In [45]:
response['answer']

'AI agents are artificial intelligence systems that integrate with databases and external tools to enhance problem-solving capabilities and adaptability. They can be built using various architectures and paradigms, such as the ReAct paradigm, to solve multistep problems. AI agents can "think" and plan after each action taken, using Think-Act-Observe loops to iteratively improve upon responses.'

In [46]:
print(rag_chain.invoke({'input':'describe more about it'})['answer'])

The AI agent is designed to assist a user in planning their surfing trip in Greece by predicting the best week for optimal surfing conditions. To achieve this, the agent gathers information from external databases and communicates with specialized agents to learn about ideal weather conditions, which include high tides, sunny weather, and minimal rain. The agent then combines this information to identify patterns and make a prediction, which is presented to the user, and subsequently refined through feedback mechanisms to improve its performance and adapt to user preferences.


In [47]:
print(rag_chain.invoke({'input':'What is ReWoo'})['answer'])

ReWOO (reasoning without observation) is a method that eliminates the dependence on tool outputs for action planning, allowing agents to plan upfront and avoid redundant tool usage. This approach involves anticipating which tools to use upon receiving a user's prompt and planning ahead to reduce token usage and computational complexity. ReWOO is desirable from a human-centered perspective as it allows users to confirm the plan before it is executed.


In [48]:
print(rag_chain.invoke({'input':'describe more about it'})['answer'])

The AI agent is designed to assist a user in planning their surfing trip to Greece by predicting the best week for surfing based on weather conditions. The agent gathers information from external databases and specialized agents to learn about optimal surfing conditions, such as high tides and sunny weather with little rain. The agent then combines this information to make a prediction and presents it to the user, while also using feedback mechanisms to improve its accuracy and adjust to user preferences for future goals.


In [49]:
#Adding Chat History

In [50]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Rewrite the user’s latest question so it can be understood on its own, "
    "without relying on prior chat history. Include any necessary context from "
    "the conversation. Do NOT answer the question—only reformulate it."
)

contextualize_q_prompt= ChatPromptTemplate.from_messages(
    [
        ('system',contextualize_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)

In [51]:
retriever_with_history=create_history_aware_retriever(llm_model,retriever,contextualize_q_prompt)
retriever_with_history

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x12ff0de70>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk'

In [52]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ('system',System_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)

In [53]:
question_answer_chain=create_stuff_documents_chain(llm_model,qa_prompt)
rag_chain=create_retrieval_chain(retriever_with_history,question_answer_chain)

In [54]:
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response1['answer'])

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Self-reflection in AI agents refers to the process of evaluating and improving their own performance and responses. AI agents use feedback mechanisms, such as human-in-the-loop (HITL) and other AI agents, to learn from their experiences and adjust to user preferences. Through self-reflection, AI agents can refine their decision-making and prediction capabilities, enabling them to become more accurate and effective over time.
Self-reflection in AI agents is the process of evaluating their own performance and responses. It involves using feedback mechanisms to learn from experiences and adjust to user preferences. This process is also referred to as "learning and reflection" in the context of AI agents.


In [55]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Self-reflection in AI agents refers to the process of evaluating and improving their own performance and responses. AI agents use feedback mechanisms, such as human-in-the-loop (HITL) and other AI agents, to learn from their experiences and adjust to user preferences. Through self-reflection, AI agents can refine their decision-making and prediction capabilities, enabling them to become more accurate and effective over time.', additional_kwargs={}, response_metadata={})]

In [56]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

In [57]:
store={}

def Get_Session_History(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

conversational_rag_chain= RunnableWithMessageHistory(
    rag_chain,
    Get_Session_History,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [58]:
from langsmith import uuid7

session_id = uuid7()

In [59]:
response=conversational_rag_chain.invoke({'input':"What is ReWoo?"},config={"configurable": {"session_id": session_id}},)
response

{'input': 'What is ReWoo?',
 'chat_history': [],
 'context': [Document(id='fe092ccd-a365-44b8-ac24-88ac995e3eb8', metadata={}, page_content="ReWOO (reasoning without observation)\nThe ReWOO method, unlike ReAct, eliminates the dependence on tool outputs for action planning. Instead, agents plan upfront. Redundant tool usage is avoided by anticipating which tools to use upon receiving the initial prompt from the user. This approach is desirable from a human-centered perspective because the user can confirm the plan before it is executed.\nThe ReWOO workflow is made up of three modules. In the planning module, the agent anticipates its next steps given a user's prompt. The next stage entails collecting the outputs produced by calling these tools. Lastly, the agent pairs the initial plan with the tool outputs to formulate a response. This planning ahead can greatly reduce token usage and computational complexity and the repercussions of intermediate tool failure.5\n Types of AI agents"),


In [60]:
response['answer']

'ReWOO (Reasoning Without Observation) is a method that eliminates dependence on tool outputs for action planning, instead planning upfront and anticipating which tools to use. This approach allows agents to plan ahead, reducing token usage and computational complexity. ReWOO is a desirable method from a human-centered perspective as it enables users to confirm the plan before execution.'

In [61]:
response=conversational_rag_chain.invoke({'input':"What was the previous question I asked"},config={"configurable": {"session_id": session_id}},)
response['answer']

'The previous question you asked was "What is ReWoo?"'

In [62]:
response=conversational_rag_chain.invoke({'input':"Tell more detailed description about it"},config={"configurable": {"session_id": session_id}},)
print(response['answer'])

ReWOO (Reasoning Without Observation) is a method used by AI agents to plan their actions upfront, without relying on the outputs of various tools. This approach involves anticipating which tools to use and in what order, based on the initial prompt or input received from the user. The ReWOO workflow consists of three modules: 

1. Planning module: The agent anticipates its next steps given a user's prompt, creating an initial plan of action.
2. Tool output collection: The agent collects the outputs produced by calling the anticipated tools.
3. Response formulation: The agent pairs the initial plan with the tool outputs to formulate a response to the user.

By planning ahead, ReWOO reduces the need for redundant tool usage, decreases token usage, and minimizes computational complexity. Additionally, it allows users to confirm the plan before it is executed, making it a desirable approach from a human-centered perspective. This method is particularly useful for complex tasks that requir

In [63]:
response=conversational_rag_chain.invoke({'input':"What is agentic ai?"},config={"configurable": {"session_id": session_id}},)
print(response['answer'])

I don't know what "agentic AI" specifically refers to, as it is not a standard term in the provided context. However, based on the text, it appears to be related to AI systems that are autonomous and can make decisions, but I would need more information to provide a accurate description. The term "agentic" is mentioned in the context of "agentic AI system", which is designed and trained by a team of developers.


In [64]:
response=conversational_rag_chain.invoke({'input':"What was the first question I asked"},config={"configurable": {"session_id": session_id}},)
response['answer']

'The first question you asked was "What is ReWoo?"'

In [65]:
store

{UUID('019abed1-1996-7402-b321-65643950fda5'): InMemoryChatMessageHistory(messages=[HumanMessage(content='What is ReWoo?', additional_kwargs={}, response_metadata={}), AIMessage(content='ReWOO (Reasoning Without Observation) is a method that eliminates dependence on tool outputs for action planning, instead planning upfront and anticipating which tools to use. This approach allows agents to plan ahead, reducing token usage and computational complexity. ReWOO is a desirable method from a human-centered perspective as it enables users to confirm the plan before execution.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was the previous question I asked', additional_kwargs={}, response_metadata={}), AIMessage(content='The previous question you asked was "What is ReWoo?"', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell more detailed description about it', additional_kwargs={}, response_metadata={}), AIMessage(content="ReWOO (Reasoning With